# Write a Table Directly from Row Data

<div style="display: inline-flex; align-items: center; gap: 10px;">
        <a href="https://colab.research.google.com/github/3lc-ai/notebook-examples/blob/main/write-mammoth-table.ipynb"
        target="_blank"
            style="background-color: transparent; text-decoration: none; display: inline-flex; align-items: center;
            padding: 5px 10px; font-family: Arial, sans-serif;"> <img
            src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" style="height: 30px;
            vertical-align: middle;box-shadow: none;"/>
        </a> <a href="https://github.com/3lc-ai/notebook-examples/blob/main/write-mammoth-table.ipynb"
            style="text-decoration: none; display: inline-flex; align-items: center; background-color: #ffffff; border:
            1px solid #d1d5da; border-radius: 8px; padding: 2px 10px; color: #333; font-family: Arial, sans-serif;">
            <svg aria-hidden="true" focusable="false" role="img" class="octicon octicon-mark-github" viewBox="0 0 16 16"
            width="20" height="20" fill="#333"
            style="display:inline-block;user-select:none;vertical-align:text-bottom;overflow:visible; margin-right:
            8px;">
                <path d="M8 0c4.42 0 8 3.58 8 8a8.013 8.013 0 0 1-5.45 7.59c-.4.08-.55-.17-.55-.38 0-.27.01-1.13.01-2.2
                0-.75-.25-1.23-.54-1.48 1.78-.2 3.65-.88 3.65-3.95 0-.88-.31-1.59-.82-2.15.08-.2.36-1.02-.08-2.12 0
                0-.67-.22-2.2.82-.64-.18-1.32-.27-2-.27-.68 0-1.36.09-2 .27-1.53-1.03-2.2-.82-2.2-.82-.44 1.1-.16
                1.92-.08 2.12-.51.56-.82 1.28-.82 2.15 0 3.06 1.86 3.75 3.64 3.95-.23.2-.44.55-.51
                1.07-.46.21-1.61.55-2.33-.66-.15-.24-.6-.83-1.23-.82-.67.01-.27.38.01.53.34.19.73.9.82 1.13.16.45.68
                1.31 2.69.94 0 .67.01 1.3.01 1.49 0 .21-.15.45-.55.38A7.995 7.995 0 0 1 0 8c0-4.42 3.58-8 8-8Z"></path>
            </svg> <span style="vertical-align: middle; color: #333;">Open in GitHub</span>
        </a>
</div>

In this sample, we will write a `tlc.Table` directly by adding rows from a JSON file one by one.

We will use a `tlc.TableWriter` instance to build up the table, and then write it to a file.

The written table will be of type `tlc.TableFromParquet`, and the table data will be backed by a parquet file.

In [ ]:
PROJECT_NAME = "Table Writer Examples"
DATASET_NAME = "Mammoth"
TABLE_NAME = "mammoth-10k"
TLC_PUBLIC_EXAMPLES_DEVELOPER_MODE = True
INSTALL_DEPENDENCIES = False

In [ ]:
%%capture
if INSTALL_DEPENDENCIES:
    %pip --quiet install torch --index-url https://download.pytorch.org/whl/cu118
    %pip --quiet install torchvision --index-url https://download.pytorch.org/whl/cu118
    %pip --quiet install tlc

In [ ]:
import requests
import tlc

## Download Source Data

We will use the 3D mammoth data as an example. This is a popular toy dataset commonly used in the dimensionality
reduction literature.

The original data can be found in the [PaCMAP](https://github.com/YingfanWang/PaCMAP/blob/master/data/mammoth_3d.json)
github repository.

In [ ]:
response = requests.get("https://raw.githubusercontent.com/YingfanWang/PaCMAP/master/data/mammoth_3d.json")
input_data = response.json()

In [ ]:
# The input data is represented as a JSON list of lists, where each sublist is a 3D point:
print(type(input_data))
print(len(input_data))
print(len(input_data[0]))

## Writing the Table

We construct a `TableWriter`, which will determine the URL and the schema of the table we want to write. In our case,
the table will contain a single column of 3-vectors.

In [ ]:
column_name = "points"

table_writer = tlc.TableWriter(
    dataset_name=DATASET_NAME,
    project_name=PROJECT_NAME,
    table_name=TABLE_NAME,
    if_exists="overwrite",
    column_schemas={column_name: tlc.FloatVector3Schema("3D Points")},
)

In [ ]:
# Next we add the data to the table row by row.
for point in input_data:
    table_writer.add_row({column_name: point})

# Finally, we flush the table writer to ensure that all data is written to disk.
table = table_writer.finalize()

In [ ]:
# Inspect the first row of the table:
table[0]

## Next Steps

The written table can now be viewed in the 3LC Dashboard.

Some ideas for further exploration:
- Visualize the data in a scatter plot
- Apply dimensionality reduction to the data
- Segment the data into clusters